In [13]:
from utils.visuals import *
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
def generate_erdos_renyi_graphs(n_graphs, n_nodes, p):
    """Generate n_graphs Erdos-Renyi graphs with n_nodes and edge probability p."""
    graphs = []
    for _ in range(n_graphs):
        G = nx.erdos_renyi_graph(n_nodes, p)
        adj_matrix = nx.to_numpy_array(G)
        graphs.append(adj_matrix)
    return graphs

In [19]:
__N_NODES__ = 10
__N_TRAIN__ = 300 #should be divisible by two 

x_train = [*generate_erdos_renyi_graphs(int(__N_TRAIN__/2), __N_NODES__, 0.4), *generate_erdos_renyi_graphs(int(__N_TRAIN__/2), __N_NODES__, 0.8)]
x_train = np.array(x_train)

In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model

# Split into train and validation sets
train_data, val_data = train_test_split(x_train, test_size=0.2, random_state=42)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices(val_data).batch(BATCH_SIZE)

# Define GCN layer
class GraphConvLayer(layers.Layer):
    def __init__(self, units, activation=None):
        super(GraphConvLayer, self).__init__()
        self.units = units
        self.activation = activation
        self.weight = None
        
    def build(self, input_shape):
        features_shape = input_shape[0]
        input_dim = features_shape[-1]
        
        # Initialize weights
        self.weight = self.add_weight(
            shape=(input_dim, self.units),
            initializer="glorot_uniform",
            trainable=True,
            name="weight"
        )
        
    def call(self, inputs):
        # Unpack inputs
        features, adj_matrix = inputs
        
        # Add self-connections to adjacency matrix
        batch_size = tf.shape(adj_matrix)[0]
        n_nodes = tf.shape(adj_matrix)[1]
        
        # Add self-loops: A' = A + I
        eye = tf.eye(n_nodes, batch_shape=[batch_size])
        adj_with_self = adj_matrix + eye
        
        # Compute degree matrix
        row_sum = tf.reduce_sum(adj_with_self, axis=-1)
        deg_inv_sqrt = tf.pow(row_sum, -0.5)
        deg_inv_sqrt = tf.where(tf.math.is_inf(deg_inv_sqrt), tf.zeros_like(deg_inv_sqrt), deg_inv_sqrt)
        deg_inv_sqrt = tf.linalg.diag(deg_inv_sqrt)
        
        # Normalized adjacency: D^(-1/2) * A' * D^(-1/2)
        norm_adj = tf.matmul(tf.matmul(deg_inv_sqrt, adj_with_self), deg_inv_sqrt)
        
        # Graph convolution: D^(-1/2) * A' * D^(-1/2) * X * W
        output = tf.matmul(norm_adj, features)
        output = tf.matmul(output, self.weight)
        
        # Apply activation if specified
        if self.activation is not None:
            output = self.activation(output)
            
        return output

# Create the VGAE model with GCN layers
class VGAE(Model):
    def __init__(self, n_nodes, hidden_dim, latent_dim):
        super(VGAE, self).__init__()
        self.n_nodes = n_nodes
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        # Initial node features (eye matrix as one-hot encoding of nodes)
        self.node_features = tf.Variable(
            initial_value=tf.eye(n_nodes),
            trainable=False,
            dtype=tf.float32,
            name="node_features"
        )
        
        # GCN Encoder layers
        self.gc1 = GraphConvLayer(hidden_dim, activation=tf.nn.relu)
        self.gc2_mean = GraphConvLayer(latent_dim)
        self.gc2_logvar = GraphConvLayer(latent_dim)
        
        # Decoder
        self.decoder = layers.Dense(n_nodes, activation='sigmoid')
        
    def encode(self, adj):
        # Expand node features for batch dimension
        batch_size = tf.shape(adj)[0]
        features = tf.tile(tf.expand_dims(self.node_features, 0), [batch_size, 1, 1])
        
        # First GCN layer
        h1 = self.gc1([features, adj])
        
        # Second GCN layer for mean and log variance
        z_mean = self.gc2_mean([h1, adj])
        z_log_var = self.gc2_logvar([h1, adj])
        
        return z_mean, z_log_var
    
    def reparameterize(self, mean, log_var):
        epsilon = tf.random.normal(shape=tf.shape(mean))
        return mean + tf.exp(0.5 * log_var) * epsilon
    
    def decode(self, z):
        # Inner product decoder
        logits = tf.matmul(z, z, transpose_b=True)
        # Apply sigmoid to get probabilities
        adj_prob = tf.sigmoid(logits)
        return adj_prob
    
    def call(self, inputs):
        z_mean, z_log_var = self.encode(inputs)
        z = self.reparameterize(z_mean, z_log_var)
        reconstructed = self.decode(z)
        return reconstructed, z_mean, z_log_var

# Initialize the model
vgae = VGAE(n_nodes=__N_NODES__, hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM)

# Define the loss function
def vgae_loss(model, x):
    # Forward pass
    x_reconstructed, z_mean, z_log_var = model(x)
    
    # Mask for removing diagonal elements (self-loops)
    batch_size = tf.shape(x)[0]
    n_nodes = tf.shape(x)[1]
    mask = 1.0 - tf.eye(n_nodes, batch_shape=[batch_size])
    
    # Reconstruction loss (binary cross-entropy with masking)
    reconstruction_loss = tf.nn.sigmoid_cross_entropy_with_logits(
        logits=tf.math.log(x_reconstructed / (1 - x_reconstructed + 1e-10)),
        labels=x
    )
    reconstruction_loss = mask * reconstruction_loss
    reconstruction_loss = tf.reduce_sum(reconstruction_loss) / tf.reduce_sum(mask)
    
    # KL divergence
    kl_loss = -0.5 / (n_nodes * batch_size) * tf.reduce_sum(
        1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    )
    
    # Total loss
    total_loss = reconstruction_loss + kl_loss
    
    return total_loss, reconstruction_loss, kl_loss

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Training function
@tf.function
def train_step(model, x, optimizer):
    with tf.GradientTape() as tape:
        total_loss, reconstruction_loss, kl_loss = vgae_loss(model, x)
    
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return total_loss, reconstruction_loss, kl_loss

# Training loop
train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    # Training
    epoch_train_loss = 0
    epoch_train_rec_loss = 0
    epoch_train_kl_loss = 0
    n_batches = 0
    
    for batch in train_dataset:
        total_loss, reconstruction_loss, kl_loss = train_step(vgae, batch, optimizer)
        epoch_train_loss += total_loss
        epoch_train_rec_loss += reconstruction_loss
        epoch_train_kl_loss += kl_loss
        n_batches += 1
    
    epoch_train_loss /= n_batches
    epoch_train_rec_loss /= n_batches
    epoch_train_kl_loss /= n_batches
    
    # Validation
    epoch_val_loss = 0
    epoch_val_rec_loss = 0
    epoch_val_kl_loss = 0
    n_batches = 0
    
    for batch in val_dataset:
        total_loss, reconstruction_loss, kl_loss = vgae_loss(vgae, batch)
        epoch_val_loss += total_loss
        epoch_val_rec_loss += reconstruction_loss
        epoch_val_kl_loss += kl_loss
        n_batches += 1
    
    epoch_val_loss /= n_batches
    epoch_val_rec_loss /= n_batches
    epoch_val_kl_loss /= n_batches
    
    train_losses.append(epoch_train_loss.numpy())
    val_losses.append(epoch_val_loss.numpy())
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{EPOCHS}, "
              f"Train Loss: {epoch_train_loss:.4f}, "
              f"Val Loss: {epoch_val_loss:.4f}, "
              f"Train Rec Loss: {epoch_train_rec_loss:.4f}, "
              f"Train KL Loss: {epoch_train_kl_loss:.4f}")

# Plot training curve
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('VGAE Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()



/Users/aldric-l/Library/Python/3.11/lib/python/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'vgae_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


TypeError: in user code:

    File "/var/folders/qc/fbm2wz190nqd87d2mq7l90_80000gn/T/ipykernel_71628/922193024.py", line 159, in train_step  *
        total_loss, reconstruction_loss, kl_loss = vgae_loss(model, x)
    File "/var/folders/qc/fbm2wz190nqd87d2mq7l90_80000gn/T/ipykernel_71628/922193024.py", line 135, in vgae_loss  *
        reconstruction_loss = tf.nn.sigmoid_cross_entropy_with_logits(

    TypeError: Input 'y' of 'Mul' Op has type float64 that does not match type float32 of argument 'x'.
